В рамках этого pet-проекта покажу как использовать API музея, позволяющий получать данные о музейных коллекциях, с использованием технологии многопоточности для повышения производительности, записывать в Google Sheet и отправлять полученныe данныe на VK и Telegram. Это задание может пригодиться для большего понимания того, какими способами можно собирать, обрабатывать, отправлять и записывать данные, например, для собственного проекта или другой рабочей задачи. Также вы можете более глубоко изучить документацию, чтобы потом исследовать другие интересные для себя вопросы по полученным данным :)

**Описание API**
Документация: https://metmuseum.github.io

Ключ: для использования выбранного API (на момент написания задания) ключ не требуется :)

Всего есть 4 основных раздела:

* Объекты (Objects)
* Объект (Object) – данные об объекте (например, о картине)
* Департаменты (Departments) –  данные о департаменте/разделе (например, "Современное искусство") 
* Поиск (Search) – используется для поиска

В рамках задачи поработаем с блоком `Search`.

С более подробным описанием каждого доступного параметра можно ознакомиться в документации.

## Допустим, нам поставили задачу выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и записать id объектов в cat_ids. Нужно получить данные о первой 1000 объектов из списка, а именно – следующие параметры:

* objectID – id объекта
* title – название
* artistDisplayName – автор
* department – департамент
* objectBeginDate – дата (начало)
* objectEndDate – дата (конец)
* period – название периода
* objectName – название/категория объекта
* culture – культура

**PS**:
Сам запрос к API должен состоять из нескольких параметров. Пример обращения к поиску (Search):**
```python
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=QUERY')
res = r.json()
```
**где QUERY – ключевое слово.* Передавать несколько параметров можно через "&"

In [1]:
# 1. Импорт нужных библиотек
import requests #Для HTTP запросов
import pandas as pd #Для создания датафрейма и манипуляций с ним
import numpy as np
from concurrent.futures import ThreadPoolExecutor #Для многопоточности

In [2]:
# 2. Создаем список необходимых параметров
suitable_sections = ['objectID', 'title', 'artistDisplayName',
                     'department', 'objectBeginDate', 'objectEndDate',
                     'period', 'objectName', 'culture']

In [3]:
# 3. Получаем идентификаторы первой 1000 объектов 
response = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
response.raise_for_status()  # Убедимся, что запрос выполнен успешно
cat_ids = response.json().get('objectIDs', [])[:1000]

# Если список пустой, сразу завершаем
if not cat_ids:
    raise ValueError("No object IDs found for the specified query.")

In [4]:
# 4. Функция для обработки одного объекта
def fetch_object(obj_id):
    try:
        r = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{obj_id}')
        r.raise_for_status()  # Проверяем статус ответа
        obj_temp = r.json()
        # Возвращаем только нужные поля, которые есть в нашем списке параметров
        return {i: obj_temp.get(i) for i in suitable_sections}
    except Exception as e:
        # В случае ошибки возвращаем None или логируем ошибку
        print(f"Error fetching object {obj_id}: {e}")
        return None

In [5]:
# 5. Многопоточная загрузка данных
# Выполнение зависит от количества потоков, на 10 потоках, например занимает примерно 1м
with ThreadPoolExecutor(max_workers=10) as executor:  # Параллельно загружаем данные в список result
    results = list(executor.map(fetch_object, cat_ids))

In [6]:
# 6. Удаляем пустые результаты (ошибки) и записываем в наш лист
results = [result for result in results if result is not None]

In [7]:
# 7. Преобразуем данные в DataFrame
final_df = pd.DataFrame(results)

In [8]:
# Просмотр полученных данных
final_df.head(3) 

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,49698,Cat,Zhang Yuguang,Asian Art,1900,1968,,Folding fan mounted as an album leaf,China
1,49470,Cat,Wang Li,Asian Art,1813,1879,Qing dynasty (1644–1911),Hanging scroll,China
2,36221,Cat,Wang Yun,Asian Art,1900,1933,,Hanging scroll,China


### Запись датафрейма в Google Sheet

In [9]:
# 1. Импортируем дополнительные библиотеки для работы с Google Sheet
import gspread
from gspread_dataframe import set_with_dataframe

# 2. Авторизация (как создать сервисный аккаунт google https://support.google.com/a/answer/7378726?hl=ru)
gc = gspread.service_account(filename='keen-scion-11111-s1-622df2b2c883.json') #Путь до файла с авторизацией

# 3. Создаем таблицу по её имени
sh = gc.create('new_tab')
                #Или
# Открываем существующую таблицу по её имени, если уже создали
# sh = gc.open('new_tab')

# 4. Делаем таблицу доступной для пользователя, если нужно кому-то отправить для просмотра или редактирования
# !!!Обязательно открывайте доступ для своей почты, с которой будете просматривать или редактировать
sh.share('ex@gmail.com', perm_type='user', role='writer')

# 5. Открываем первый рабочий лист
worksheet = sh.sheet1  # Получаем первый лист, если нужно
                #Или
# Получаем доступ к листу по имени
# worksheet = sh.worksheet('Sheet1')
                #Или
# Добавляем новый лист с именем "Ваше имя страницы" если нужно
# worksheet = sh.add_worksheet(title="Ваше имя страницы", rows="100", cols="20")

# 6. Проверяйте наличие DataFrame и его содержимого
if not final_df.empty:
    # Записываем DataFrame в Google Sheets
    set_with_dataframe(worksheet, final_df, include_index=False)

    print("Данные успешно записаны в Google Sheets!")
else:
    print("DataFrame пуст, данные не записаны.")

Данные успешно записаны в Google Sheets!


In [10]:
# Получаем ссылку нашей таблицы для быстрого доступа
link = sh.url

### Отправка полученной ссылки Google Sheet с данными на VK и Telegram 
**Рассылка данных через мессенджеры позволяет сотрудникам легко и удобно получать необходимую информацию ежедневно.**

#### Telegram

In [11]:
# Ваш токен бота (как создать бота https://core.telegram.org/bots/tutorial)
TOKEN = '6476859971:AAE-ZDtx'
# ID пользователя или беседы, куда хотите отправить сообщение
CHAT_ID = '4169987'  # Замените на ID вашего рабочего чата с коллегами

# Данные для отправки
message = 'Здравствуйте, коллеги! Ежедневная рассылка данных:\n\n' + link

def send_telegram_message(token, chat_id, message):
    # Формируем URL для отправки сообщения
    url = f'https://api.telegram.org/bot{token}/sendMessage'
    
    # Параметры запроса
    params = {
        'chat_id': chat_id,
        'text': message
    }
    try:
        # Отправляем POST-запрос
        response = requests.post(url, params=params)
        # Проверяем статус ответа
        if response.status_code == 200:
            print('Сообщение отправлено успешно!')
        else:
            print(f'Ошибка при отправке сообщения: {response.status_code}, {response.text}')
    except requests.RequestException as e:
        print(f'Ошибка подключения к Telegram API: {e}')

# Вызов функции
send_telegram_message(TOKEN, CHAT_ID, message)

Сообщение отправлено успешно!


### VK

In [12]:
# Импортируем дополнительные библиотеки для работы с VK
import vk_api

# Токен доступа
TOKEN = 'vk1.a.Bbt7bmA3JLfnayNFkl9llSKi0JaPKQJ_MuA-VjXkZCr_t_QiqnfDfK8_cYJQmK3SmKQr-hBDzWyLT60c18paFoj7svfBvStoRDnj4NiJxK7is-A'
# ID пользователя или беседы, куда хотите отправить сообщение
CHAT_ID = 1 # Замените на ID вашего рабочего чата с коллегами

# Данные для отправки
message = 'Здравствуйте, коллеги! Ежедневная рассылка данных:\n\n' + link

def send_vk_message(token, chat_id, message):
    try:
        vk_session = vk_api.VkApi(token=token)
        vk = vk_session.get_api()
        
        vk.messages.send(
            chat_id=chat_id,
            random_id=np.random.randint(0, 2**31 - 1), # Уникальный ID сообщения (любое число, чтобы исключить дубли)
            message=message,
        )
        print("Сообщение отправлено успешно!")
    except vk_api.exceptions.ApiError as e:
        print(f"Ошибка VK API: {e}")
    except Exception as ex:
        print(f"Неизвестная ошибка: {ex}")

send_vk_message(TOKEN, CHAT_ID, message)

Сообщение успешно отправлено.
